In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_squared_error

# === Load Stock Data ===
df = pd.read_csv("/content/drive/MyDrive/BAJFINANCE.csv")  # You can change to any other stock file here
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date')

# === Feature Engineering ===
df['Prev_Close'] = df['Close'].shift(1)
df['Prev_Volume'] = df['Volume'].shift(1)
df_model = df[['Close', 'Prev_Close', 'Prev_Volume']].dropna()

X = df_model[['Prev_Close', 'Prev_Volume']]
y = df_model['Close']

# === Train/Test Split and Scaling ===
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.2)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# === Train Models ===
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "XGBoost": XGBRegressor(n_estimators=100, random_state=42, verbosity=0)
}

best_model = None
best_score = float('-inf')

print("\n🔍 Model Performance on Test Data:\n")
for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    preds = model.predict(X_test_scaled)
    r2 = r2_score(y_test, preds)
    mse = mean_squared_error(y_test, preds)
    print(f"{name} -> R² Score: {r2:.4f} | MSE: {mse:.2f}")
    if r2 > best_score:
        best_model = model
        best_score = r2

print(f"\n✅ Best Model Selected: {type(best_model).__name__} (R² Score: {best_score:.4f})")

# === Interactive User Input Loop ===
while True:
    print("\n📊 Predict Next-Day Stock Price")
    try:
        prev_close = float(input("Enter previous day's closing price (e.g., 250.50): "))
        prev_volume = float(input("Enter previous day's volume (e.g., 1200000): "))

        user_features = scaler.transform([[prev_close, prev_volume]])
        prediction = best_model.predict(user_features)[0]

        print(f"💰 Predicted next-day closing price: ₹{prediction:.2f}")
    except Exception as e:
        print(f"⚠️ Error: {e}")

    cont = input("\n🔁 Want to predict again? (y/n): ").strip().lower()
    if cont != 'y':
        print("👋 Exiting the predictor. Have a great day!")
        break



🔍 Model Performance on Test Data:

Linear Regression -> R² Score: 0.9948 | MSE: 7240.47
Random Forest -> R² Score: 0.9932 | MSE: 9519.39
XGBoost -> R² Score: 0.9872 | MSE: 17851.41

✅ Best Model Selected: LinearRegression (R² Score: 0.9948)

📊 Predict Next-Day Stock Price


In [4]:
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive
